In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Generating synthetic dataset: configuration

In [38]:
N = 2000                              # Number of samples
interval_minutes = 10                 # Sensor frequency (every 10 minutes)
start_time = datetime(2024, 1, 1, 0, 0)

# Environmental baseline values
soil_moisture = 45.0                 # % VWC
temperature = 24.0                   # °C
humidity = 55.0                      # %
rainfall_forecast = 0.0              # mm (24h)

# Optimal thresholds (You can adjust for your greenhouse)
OPTIMAL_MIN_MOISTURE = 40
OPTIMAL_MAX_MOISTURE = 60
RAINFALL_THRESHOLD = 5               # If forecasted rain < 5 mm → irrigation needed

# Helper functions
def evaporation(temp, humidity):
    """Simple evaporation model: hotter + drier = more evaporation."""
    return max(0, 0.15 + 0.02 * (temp - 20) - 0.01 * (humidity - 50))

def rainfall_effect(rain):
    """Rain increases soil moisture slightly."""
    return 0.5 * rain

# Generating synthetic dataset: Data generation loop

In [39]:
timestamps = []
soil_list = []
temp_list = []
hum_list = []
rain_list = []
irrigation_list = []

current_time = start_time

for i in range(N):

    # 1) GENERATE TIMESTAMP
    timestamps.append(current_time)
    current_time += timedelta(minutes=interval_minutes)

    # 2) UPDATE ENV VARs
    temperature += np.random.uniform(-0.2, 0.2)
    humidity += np.random.uniform(-0.3, 0.3)

    # Daily rainfall forecast randomness
    if i % int(24*60/interval_minutes) == 0:
        rainfall_forecast = max(0, np.random.normal(3, 2))
    
    # Soil moisture dynamic 
    soil_moisture = soil_moisture \
                     - evaporation(temperature, humidity) \
                     + rainfall_effect(rainfall_forecast) \
                     + np.random.uniform(-0.1, 0.1)  # tiny noise

    soil_moisture = np.clip(soil_moisture, 10, 80)

    # 3) CONTROL LOGIC
    if soil_moisture < OPTIMAL_MIN_MOISTURE and rainfall_forecast < RAINFALL_THRESHOLD:
        irrigation = "HIGH"
    elif OPTIMAL_MIN_MOISTURE <= soil_moisture <= OPTIMAL_MAX_MOISTURE:
        irrigation = "MEDIUM"
    else:
        irrigation = "LOW"

    # Append data
    soil_list.append(soil_moisture)
    temp_list.append(temperature)
    hum_list.append(humidity)
    rain_list.append(rainfall_forecast)
    irrigation_list.append(irrigation)
       
# print(soil_list)
# print(temp_list)
# print(hum_list)
# print(rain_list)
# print(irrigation_list)

# Create dataframe and save it to csv

In [40]:
df = pd.DataFrame({
    "timestamp": timestamps,
    "soil_moisture": soil_list,
    "temperature": temp_list,
    "humidity": hum_list,
    "rainfall_forecast": rain_list,
    "irrigation_action": irrigation_list
})

# Save to CSV
df.to_csv("synthetic_greenhouse_dataset.csv", index=False)

df.head()

,timestamp,soil_moisture,temperature,humidity,rainfall_forecast,irrigation_action
0,2024-01-01 00:00:00,45.383476,24.032586,54.866240,1.247444,MEDIUM
1,2024-01-01 00:10:00,45.808338,23.862180,55.136054,1.247444,MEDIUM
2,2024-01-01 00:20:00,46.251169,24.042344,54.843697,1.247444,MEDIUM
3,2024-01-01 00:30:00,46.636340,24.083316,54.645452,1.247444,MEDIUM
4,2024-01-01 00:40:00,47.055346,24.230103,54.882819,1.247444,MEDIUM


# Fuzzyfication layer
## Membership functions

In [41]:
def trimf(x, a, b, c):
    
    if x<=a and x>=c:
        return 0.0

    elif a < x < b:
        return (x-a)/(b-a)
    
    elif b <= x < c:
        return (c-x)/(c-b)
    
    else:
        return 1.0

In [42]:
def trapmf(x, a, b, c, d):

    if x<=a or x>=d:
        return 0.0
    elif a < x < b:
        return (x-a)/(b-a)
    elif b <= x <= c:
        return 1.0
    elif c < x < d:
        return (d-x)/(d-c)

In [43]:
MF = {
    "soil_moisture": {
        "low":    lambda x: trapmf(x, 0, 0, 15, 30),
        "medium": lambda x: trimf(x, 15, 30, 45),
        "high":   lambda x: trapmf(x, 30, 45, 60, 60)
    },

    "temperature": {
        "cold":   lambda x: trimf(x, -5, 5, 15),
        "normal": lambda x: trimf(x, 10, 20, 30),
        "hot":    lambda x: trimf(x, 25, 35, 45),
    },

    "humidity": {
        "dry":    lambda x: trapmf(x, 0, 0, 20, 40),
        "normal": lambda x: trimf(x, 30, 50, 70),
        "humid":  lambda x: trapmf(x, 60, 80, 100, 100),
    },

    "rainfall_forecast": {
        "low":    lambda x: trapmf(x, 0, 0, 5, 15),
        "medium": lambda x: trimf(x, 10, 20, 30),
        "high":   lambda x: trapmf(x, 20, 40, 60, 60)
    }
}

In [44]:
df.head()

,timestamp,soil_moisture,temperature,humidity,rainfall_forecast,irrigation_action
0,2024-01-01 00:00:00,45.383476,24.032586,54.866240,1.247444,MEDIUM
1,2024-01-01 00:10:00,45.808338,23.862180,55.136054,1.247444,MEDIUM
2,2024-01-01 00:20:00,46.251169,24.042344,54.843697,1.247444,MEDIUM
3,2024-01-01 00:30:00,46.636340,24.083316,54.645452,1.247444,MEDIUM
4,2024-01-01 00:40:00,47.055346,24.230103,54.882819,1.247444,MEDIUM


## Applying MF to form the fuzzified dataset

In [45]:
fuzzy_df = pd.DataFrame()

for feature, mfs in MF.items():
    for mf_name, mf_func in mfs.items():
        col_name = f"{feature}_{mf_name}"
        fuzzy_df[col_name] = df[feature].apply(mf_func)

In [46]:
fuzzy_df.shape

(2000, 12)

In [55]:
fuzzy_df.head(50)

,soil_moisture_low,soil_moisture_medium,soil_moisture_high,temperature_cold,temperature_normal,temperature_hot,humidity_dry,humidity_normal,humidity_humid,rainfall_forecast_low,rainfall_forecast_medium,rainfall_forecast_high
0,0.0,1.0,1.0,1.0,0.596741,1.000000,0.0,0.756688,0.0,1.0,1.0,0.0
1,0.0,1.0,1.0,1.0,0.613782,1.000000,0.0,0.743197,0.0,1.0,1.0,0.0
2,0.0,1.0,1.0,1.0,0.595766,1.000000,0.0,0.757815,0.0,1.0,1.0,0.0
3,0.0,1.0,1.0,1.0,0.591668,1.000000,0.0,0.767727,0.0,1.0,1.0,0.0
4,0.0,1.0,1.0,1.0,0.576990,1.000000,0.0,0.755859,0.0,1.0,1.0,0.0
5,0.0,1.0,1.0,1.0,0.568585,1.000000,0.0,0.744786,0.0,1.0,1.0,0.0
6,0.0,1.0,1.0,1.0,0.572544,1.000000,0.0,0.747713,0.0,1.0,1.0,0.0
7,0.0,1.0,1.0,1.0,0.576685,1.000000,0.0,0.754203,0.0,1.0,1.0,0.0
8,0.0,1.0,1.0,1.0,0.566151,1.000000,0.0,0.764034,0.0,1.0,1.0,0.0
9,0.0,1.0,1.0,1.0,0.577376,1.000000,0.0,0.775804,0.0,1.0,1.0,0.0


# The Rule Layer

## MF columns

In [48]:
soil_mfs = ["soil_moisture_low", "soil_moisture_medium", "soil_moisture_high"]
temp_mfs = ["temperature_cold", "temperature_normal", "temperature_hot"]
humidity_mfs = ["humidity_dry", "humidity_normal", "humidity_humid"]
rainfall_mfs = ["rainfall_forecast_low", "rainfall_forecast_medium", "rainfall_forecast_high"]

## Rule combinations

In [49]:
import itertools

rules = list(itertools.product(soil_mfs, temp_mfs, humidity_mfs, rainfall_mfs))
len(rules)

81

## Rule firing strenghts

In [50]:
rule_outputs = pd.DataFrame()

for i, (s, t, h, r) in enumerate(rules):
    rule_name = f"R{i+1}_{s}_{t}_{h}_{r}"
    rule_outputs[rule_name] = fuzzy_df[s] * fuzzy_df[t] * fuzzy_df[h] * fuzzy_df[r]

In [51]:
rule_outputs.shape

(2000, 81)

# Normalization Layer

$\overline{w_i} = w_i \Big/ (\sum_{j=1}^{\infty} w_j)$

### Where:
* $w_1, w_2, w_3, ..., w_{81}$ are the firing strenghts from Rule layer

In [52]:
# Sum of rule strenghts across all 81 rules (per row)
rule_sum = rule_outputs.sum(axis=1)

# Prevents division-by-zero in case all firing strengths are zero
rule_sum = rule_sum.replace(0, 1e-6)

# Normalized firing strengths
normalized_rules = rule_outputs.div(rule_sum, axis=0)

normalized_rules.head()

,R1_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_low,R2_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_medium,R3_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_high,R4_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_low,R5_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_medium,R6_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_high,R7_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_low,R8_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_medium,R9_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_high,R10_soil_moisture_low_temperature_normal_humidity_dry_rainfall_forecast_low,...,R72_soil_moisture_high_temperature_normal_humidity_humid_rainfall_forecast_high,R73_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_low,R74_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_medium,R75_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_high,R76_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_low,R77_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_medium,R78_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_high,R79_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_low,R80_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_medium,R81_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_high
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096275,0.096275,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.095647,0.095647,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096311,0.096311,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096463,0.096463,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097012,0.097012,0.0,0.0,0.0,0.0


In [53]:
print("Any NaNs in rule_outputs?", rule_outputs.isna().any().any())
print("How many rows have sum=0?", (rule_outputs.sum(axis=1)==0).sum())

Any NaNs in rule_outputs? False
How many rows have sum=0? 0


In [57]:
normalized_rules.head(50)

,R1_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_low,R2_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_medium,R3_soil_moisture_low_temperature_cold_humidity_dry_rainfall_forecast_high,R4_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_low,R5_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_medium,R6_soil_moisture_low_temperature_cold_humidity_normal_rainfall_forecast_high,R7_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_low,R8_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_medium,R9_soil_moisture_low_temperature_cold_humidity_humid_rainfall_forecast_high,R10_soil_moisture_low_temperature_normal_humidity_dry_rainfall_forecast_low,...,R72_soil_moisture_high_temperature_normal_humidity_humid_rainfall_forecast_high,R73_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_low,R74_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_medium,R75_soil_moisture_high_temperature_hot_humidity_dry_rainfall_forecast_high,R76_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_low,R77_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_medium,R78_soil_moisture_high_temperature_hot_humidity_normal_rainfall_forecast_high,R79_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_low,R80_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_medium,R81_soil_moisture_high_temperature_hot_humidity_humid_rainfall_forecast_high
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096275,0.096275,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.095647,0.095647,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096311,0.096311,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096463,0.096463,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097012,0.097012,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097330,0.097330,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097180,0.097180,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097024,0.097024,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097422,0.097422,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096998,0.096998,0.0,0.0,0.0,0.0
